# Data Exploration for Consultancy Assessment

**Date:** 2025-01-27  
**Purpose:** Examine the structure and content of all three Excel files to understand:
- Sheet names and which sheets contain data
- Column names and data types
- Available indicators (especially ANC4 and SBA related)
- Country names and year coverage
- Data quality issues

**Author:** Data Analyst

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Working directory: {Path.cwd()}")

## 1. File 1: GLOBAL_DATAFLOW_2018-2022.xlsx (UNICEF Data)

This file contains the key health indicators we need: ANC4 and SBA data.

In [ ]:
# Load UNICEF data
unicef_file = Path("../01_raw_data/GLOBAL_DATAFLOW_2018-2022.xlsx")
print(f"Loading: {unicef_file}")
print(f"File exists: {unicef_file.exists()}")

# Read the data
unicef_df = pd.read_excel(unicef_file, sheet_name='Unicef data')

print(f"\nDataset shape: {unicef_df.shape}")
print(f"Time period: {unicef_df['TIME_PERIOD'].min()} - {unicef_df['TIME_PERIOD'].max()}")

In [ ]:
# Display basic information about the dataset
print("Column names:")
for i, col in enumerate(unicef_df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nData types:")
print(unicef_df.dtypes)

In [ ]:
# Examine the indicators available
indicators = unicef_df['Indicator'].unique()
print(f"Number of unique indicators: {len(indicators)}")
print("\nIndicators:")
for i, indicator in enumerate(indicators, 1):
    print(f"{i}. {indicator}")

# Count records by indicator
indicator_counts = unicef_df['Indicator'].value_counts()
print("\nRecords per indicator:")
print(indicator_counts)

In [ ]:
# Visualize indicator distribution
plt.figure(figsize=(12, 6))
indicator_counts.plot(kind='bar')
plt.title('Number of Records by Indicator')
plt.xlabel('Indicator')
plt.ylabel('Number of Records')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Show distribution by year
plt.figure(figsize=(10, 6))
year_counts = unicef_df['TIME_PERIOD'].value_counts().sort_index()
year_counts.plot(kind='bar')
plt.title('Number of Records by Year')
plt.xlabel('Year')
plt.ylabel('Number of Records')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Examine geographic areas
areas = unicef_df['Geographic area'].unique()
print(f"Number of geographic areas: {len(areas)}")
print("\nFirst 20 geographic areas:")
for i, area in enumerate(areas[:20], 1):
    print(f"{i:2d}. {area}")

# Identify regional vs country entries
regional_areas = [area for area in areas if '(' in area or area in ['Africa', 'Asia', 'Europe', 'World']]
print(f"\nPotential regional aggregates: {len(regional_areas)}")
print("Examples:", regional_areas[:10])

In [ ]:
# Focus on 2022 data
data_2022 = unicef_df[unicef_df['TIME_PERIOD'] == 2022]
print(f"2022 data records: {len(data_2022)}")
print(f"Geographic areas with 2022 data: {len(data_2022['Geographic area'].unique())}")

# Break down by indicator for 2022
print("\n2022 data by indicator:")
indicator_2022 = data_2022['Indicator'].value_counts()
print(indicator_2022)

# Show sample of 2022 data
print("\nSample of 2022 data:")
display(data_2022[['Geographic area', 'Indicator', 'OBS_VALUE', 'Sex']].head(10))

In [ ]:
# Analyze ANC4 and SBA data specifically
anc4_data = unicef_df[unicef_df['Indicator'].str.contains('Antenatal care 4+', na=False)]
sba_data = unicef_df[unicef_df['Indicator'].str.contains('Skilled birth attendant', na=False)]

print(f"ANC4 total records: {len(anc4_data)}")
print(f"SBA total records: {len(sba_data)}")

# 2022 data for each indicator
anc4_2022 = anc4_data[anc4_data['TIME_PERIOD'] == 2022]
sba_2022 = sba_data[sba_data['TIME_PERIOD'] == 2022]

print(f"\nANC4 countries with 2022 data: {len(anc4_2022['Geographic area'].unique())}")
print(f"SBA countries with 2022 data: {len(sba_2022['Geographic area'].unique())}")

# Show distribution of values for 2022
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

if len(anc4_2022) > 0:
    ax1.hist(anc4_2022['OBS_VALUE'].dropna(), bins=20, alpha=0.7, color='skyblue')
    ax1.set_title('ANC4 Coverage Distribution (2022)')
    ax1.set_xlabel('Coverage Percentage')
    ax1.set_ylabel('Number of Countries')

if len(sba_2022) > 0:
    ax2.hist(sba_2022['OBS_VALUE'].dropna(), bins=20, alpha=0.7, color='lightcoral')
    ax2.set_title('SBA Coverage Distribution (2022)')
    ax2.set_xlabel('Coverage Percentage')
    ax2.set_ylabel('Number of Countries')

plt.tight_layout()
plt.show()

## 2. File 2: On-track and off-track countries.xlsx (Mortality Status)

This file contains the under-5 mortality status classifications for countries.

In [ ]:
# Load mortality status data
mortality_file = Path("../01_raw_data/On-track and off-track countries.xlsx")
print(f"Loading: {mortality_file}")
print(f"File exists: {mortality_file.exists()}")

mortality_df = pd.read_excel(mortality_file, sheet_name='Sheet1')

print(f"\nDataset shape: {mortality_df.shape}")
print(f"Columns: {list(mortality_df.columns)}")

# Display first few rows
print("\nFirst 10 rows:")
display(mortality_df.head(10))

In [ ]:
# Analyze mortality status distribution
status_counts = mortality_df['Status.U5MR'].value_counts()
print("Under-5 mortality status distribution:")
print(status_counts)
print(f"\nPercentages:")
print(status_counts / len(mortality_df) * 100)

# Visualize the distribution
plt.figure(figsize=(10, 6))
status_counts.plot(kind='bar', color=['green', 'orange', 'red'])
plt.title('Under-5 Mortality Status Distribution')
plt.xlabel('Status')
plt.ylabel('Number of Countries')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Pie chart
plt.figure(figsize=(8, 8))
plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', 
        colors=['green', 'orange', 'red'])
plt.title('Under-5 Mortality Status Distribution')
plt.show()

In [ ]:
# Show sample countries by status
for status in mortality_df['Status.U5MR'].unique():
    countries = mortality_df[mortality_df['Status.U5MR'] == status]['OfficialName'].tolist()
    print(f"\n{status} ({len(countries)} countries):")
    print(f"Sample: {countries[:10]}")

## 3. File 3: WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx

This file contains demographic data including birth and population statistics.

In [ ]:
# Load WPP2022 data with special handling for headers
wpp_file = Path("../01_raw_data/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx")
print(f"Loading: {wpp_file}")
print(f"File exists: {wpp_file.exists()}")

# Check sheet names
excel_file = pd.ExcelFile(wpp_file)
print(f"Sheet names: {excel_file.sheet_names}")

# Read raw data to find header row
estimates_raw = pd.read_excel(wpp_file, sheet_name='Estimates', header=None)
print(f"\nEstimates raw shape: {estimates_raw.shape}")

# Look for header row
print("\nLooking for header row in first 20 rows:")
for i in range(min(20, len(estimates_raw))):
    non_null_count = estimates_raw.iloc[i].notna().sum()
    if non_null_count > 10:
        row_sample = estimates_raw.iloc[i].dropna().astype(str).tolist()[:5]
        print(f"Row {i} ({non_null_count} non-null): {row_sample}")

In [ ]:
# Load with proper header (row 16 based on exploration)
estimates_df = pd.read_excel(wpp_file, sheet_name='Estimates', header=16)
projections_df = pd.read_excel(wpp_file, sheet_name='Projections', header=16)

print(f"Estimates shape: {estimates_df.shape}")
print(f"Projections shape: {projections_df.shape}")

print(f"\nEstimates columns (first 10):")
print(list(estimates_df.columns[:10]))

print(f"\nProjections columns (first 10):")
print(list(projections_df.columns[:10]))

In [ ]:
# Examine geographic coverage
country_col = 'Region, subregion, country or area *'
if country_col in estimates_df.columns:
    areas_wpp = estimates_df[country_col].unique()
    print(f"Number of geographic areas in WPP2022: {len(areas_wpp)}")
    print("\nFirst 20 areas:")
    for i, area in enumerate(areas_wpp[:20], 1):
        print(f"{i:2d}. {area}")
    
    # Check for 2022 data
    if 'Year' in estimates_df.columns:
        years = estimates_df['Year'].unique()
        print(f"\nYear range in estimates: {min(years)} - {max(years)}")
        
        # Check 2022 data availability
        data_2022_wpp = estimates_df[estimates_df['Year'] == 2022]
        print(f"2022 records in estimates: {len(data_2022_wpp)}")
        print(f"Countries with 2022 data: {len(data_2022_wpp[country_col].unique())}")

In [ ]:
# Look for birth-related columns
birth_cols = [col for col in estimates_df.columns if 'birth' in col.lower() or 'born' in col.lower()]
print(f"Birth-related columns: {birth_cols}")

# Look for population columns
pop_cols = [col for col in estimates_df.columns if 'population' in col.lower() or 'pop' in col.lower()]
print(f"\nPopulation-related columns (first 10): {pop_cols[:10]}")

# Show sample data for 2022
if len(data_2022_wpp) > 0:
    print("\nSample 2022 data:")
    sample_cols = [country_col, 'Year'] + birth_cols[:3] + pop_cols[:3]
    available_cols = [col for col in sample_cols if col in data_2022_wpp.columns]
    display(data_2022_wpp[available_cols].head(10))

## 4. Country Name Comparison Across Datasets

Let's examine how country names align across the three datasets.

In [ ]:
# Extract country names from each dataset
unicef_countries = set(unicef_df['Geographic area'].unique())
mortality_countries = set(mortality_df['OfficialName'].unique())
wpp_countries = set(estimates_df[country_col].unique()) if country_col in estimates_df.columns else set()

print(f"UNICEF geographic areas: {len(unicef_countries)}")
print(f"Mortality file countries: {len(mortality_countries)}")
print(f"WPP2022 areas: {len(wpp_countries)}")

# Find overlaps
unicef_mortality_overlap = unicef_countries.intersection(mortality_countries)
unicef_wpp_overlap = unicef_countries.intersection(wpp_countries)
mortality_wpp_overlap = mortality_countries.intersection(wpp_countries)

print(f"\nUNICEF-Mortality exact matches: {len(unicef_mortality_overlap)}")
print(f"UNICEF-WPP exact matches: {len(unicef_wpp_overlap)}")
print(f"Mortality-WPP exact matches: {len(mortality_wpp_overlap)}")

# All three datasets
all_three_overlap = unicef_countries.intersection(mortality_countries).intersection(wpp_countries)
print(f"All three datasets exact matches: {len(all_three_overlap)}")

In [ ]:
# Visualize overlaps
from matplotlib_venn import venn3
import matplotlib.pyplot as plt

# Create Venn diagram (if matplotlib_venn is available)
try:
    plt.figure(figsize=(10, 8))
    venn3([unicef_countries, mortality_countries, wpp_countries], 
          ('UNICEF', 'Mortality', 'WPP2022'))
    plt.title('Country Name Overlaps Across Datasets')
    plt.show()
except ImportError:
    print("matplotlib_venn not available, showing text summary instead")
    
# Show some examples of non-matching countries
unicef_only = unicef_countries - mortality_countries - wpp_countries
mortality_only = mortality_countries - unicef_countries - wpp_countries

print(f"\nCountries only in UNICEF dataset (first 10): {list(unicef_only)[:10]}")
print(f"Countries only in Mortality dataset (first 10): {list(mortality_only)[:10]}")

## 5. Data Quality Assessment

Let's examine missing values and data quality issues.

In [ ]:
# Missing values in UNICEF data
print("Missing values in UNICEF dataset:")
missing_unicef = unicef_df.isnull().sum()
missing_unicef = missing_unicef[missing_unicef > 0].sort_values(ascending=False)
print(missing_unicef)

# Visualize missing values
if len(missing_unicef) > 0:
    plt.figure(figsize=(12, 6))
    missing_unicef.plot(kind='bar')
    plt.title('Missing Values in UNICEF Dataset')
    plt.xlabel('Columns')
    plt.ylabel('Number of Missing Values')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Check for duplicates and data consistency
print("Data consistency checks:")
print(f"\nUNICEF dataset:")
print(f"- Total records: {len(unicef_df)}")
print(f"- Duplicate rows: {unicef_df.duplicated().sum()}")
print(f"- Unique country-indicator-year combinations: {len(unicef_df[['Geographic area', 'Indicator', 'TIME_PERIOD']].drop_duplicates())}")

print(f"\nMortality dataset:")
print(f"- Total records: {len(mortality_df)}")
print(f"- Duplicate rows: {mortality_df.duplicated().sum()}")
print(f"- Unique countries: {len(mortality_df['OfficialName'].unique())}")
print(f"- Unique ISO3 codes: {len(mortality_df['ISO3Code'].unique())}")

# Check value ranges for key indicators
print(f"\nValue ranges for 2022 data:")
if len(anc4_2022) > 0:
    print(f"ANC4 coverage: {anc4_2022['OBS_VALUE'].min():.1f}% - {anc4_2022['OBS_VALUE'].max():.1f}%")
if len(sba_2022) > 0:
    print(f"SBA coverage: {sba_2022['OBS_VALUE'].min():.1f}% - {sba_2022['OBS_VALUE'].max():.1f}%")

## 6. Summary and Recommendations

Based on the exploration, here are the key findings and recommendations for data cleaning.

In [ ]:
# Create summary statistics
summary_stats = {
    'Dataset': ['UNICEF GLOBAL_DATAFLOW', 'Mortality Status', 'WPP2022 Estimates'],
    'Records': [len(unicef_df), len(mortality_df), len(estimates_df)],
    'Geographic_Areas': [len(unicef_countries), len(mortality_countries), len(wpp_countries)],
    'Has_2022_Data': ['Yes', 'No', 'Yes'],
    'Key_Indicators': ['ANC4, SBA', 'U5MR Status', 'Demographics']
}

summary_df = pd.DataFrame(summary_stats)
print("Dataset Summary:")
display(summary_df)

# Key findings
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

print(f"\n1. ANC4 AND SBA INDICATORS:")
print(f"   - ANC4 total records: {len(anc4_data)}")
print(f"   - SBA total records: {len(sba_data)}")
print(f"   - ANC4 countries with 2022 data: {len(anc4_2022['Geographic area'].unique())}")
print(f"   - SBA countries with 2022 data: {len(sba_2022['Geographic area'].unique())}")

print(f"\n2. COUNTRY STANDARDIZATION NEEDS:")
print(f"   - UNICEF file: {len(unicef_countries)} geographic areas")
print(f"   - Mortality file: {len(mortality_countries)} countries")
print(f"   - WPP2022 file: {len(wpp_countries)} areas")
print(f"   - Exact matches (UNICEF-Mortality): {len(unicef_mortality_overlap)}")

print(f"\n3. DATA QUALITY ISSUES:")
print(f"   - WPP2022 requires special header parsing (skip first 16 rows)")
print(f"   - Regional aggregates mixed with individual countries")
print(f"   - Missing values in confidence intervals and metadata")
print(f"   - Country naming inconsistencies across datasets")

print(f"\n4. RECOMMENDATIONS FOR DATA CLEANING:")
print(f"   - Create country name mapping dictionary")
print(f"   - Separate regional aggregates from individual countries")
print(f"   - Extract 2022 ANC4 and SBA data with proper country matching")
print(f"   - Parse WPP2022 file structure correctly")
print(f"   - Handle missing values appropriately")
print(f"   - Prepare merged dataset for analysis")

In [ ]:
# Save key datasets for next steps
print("Saving processed data for next steps...")

# Create output directory if it doesn't exist
output_dir = Path("../02_processed_data")
output_dir.mkdir(exist_ok=True)

# Save exploration results
exploration_summary = {
    'unicef_shape': unicef_df.shape,
    'mortality_shape': mortality_df.shape,
    'wpp_estimates_shape': estimates_df.shape,
    'anc4_2022_countries': len(anc4_2022['Geographic area'].unique()) if len(anc4_2022) > 0 else 0,
    'sba_2022_countries': len(sba_2022['Geographic area'].unique()) if len(sba_2022) > 0 else 0,
    'country_matches': len(unicef_mortality_overlap),
    'exploration_date': '2025-01-27'
}

print("Exploration completed successfully!")
print("\nNext steps:")
print("1. Create data cleaning script based on these findings")
print("2. Standardize country names across datasets")
print("3. Extract and clean 2022 ANC4 and SBA data")
print("4. Prepare merged dataset for analysis")